# Problem Statement
The data scientists at BigMart have collected **2013 sales data** for **1559 products** across **10 stores** in **different cities**. Also, certain attributes of each product and store have been defined. The aim is to **build a predictive model** and find out the sales of each product at a particular store.

Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales.


Please note that the **data may have missing values** as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.

![](https://media.giphy.com/media/VTXzh4qtahZS/giphy.gif)

## Data
We have a train (8523 observations) and test (5681 observations) dataset, the train data set has both input and output variable(s). We need to predict the sales for the test dataset.


|Variable|Description|
|---|---|
|Item_Identifier|Unique Product ID|
|Item_Weight|Weight of product|
|Item_Fat_Content|Whether the product is low fat or not|
|Item_Visability|The % of the total display area of all products in a store allocated to the particular product|
|Item_MRP|Maximum Retail Price (list price) of the product|
|Outlet_Identifier|Unique store ID|
|Outlet_Establishment_Year|The year in which store was established|
|Outlet_Size|The size of the store in terms of ground area covered|
|Outlet_Location_Type|The type of city in which the store is located|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|
|Item_Outlet_Sales|Sales of the product in a particular store. This is the outcome variable to be predicted.|

## Plan of attack
We'll explore the problem using the following steps:
1. **Hypothesis Generation**: Understand the problem by brainstorming ideas about how the outcome can be affected by possible factors
	* Store level hypothesis
	* Product level hypothesis
2. **Data Exploration**: Look at the categorical and continuous features and make inferences about the data
3. **Data Cleaning**: Check the data for missing and incorrect values, outliers. Think about strategies on how to deal with these observations.
4. **Feature Engineering (Optional)**: Modify the existing variables and create new ones
5. **Modelling**: Make predictive models on the data. Try to create a reusable pipeline for all the models.
	* Linear Regression
	* Decision Tree
	* Random Forest
	* SVR (Support Vector Regreesion)
	* XGBoost

## Additional Notes
* Combine the two datasets to do all the needed data preprocessing. After applying all the preprocessing, split the datasets again.
* Compare the model results using metrics and visualization graphics.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
%matplotlib inline

In [3]:
SEED = 42
np.random.seed(SEED)

### 1. Hypothesis Generation
#### Store Level Hypothesis
* **<span style="color:Green">Place</span>**: Stores in the urban places will have higher sales because there are more people in the urban areas rather than in the villages.
* **<span style="color:red">Population</span>**: Stores placed in locations with higher population will generate higher sales because of the demand.
* **<span style="color:red">Feedback</span>**: Stores with more positive feedback, will have higher sales because of the comfortable environment and the kind employees.
* **<span style="color:red">Competitors</span>**: Stores which have competitors nearby will generate lower sales because of more competition.
* **<span style="color:red">Marketing</span>**: Stores which have good advertising (TV ads and paper catalogs) will generate higher sales.
* **<span style="color:Green">Location</span>**: Stores which are in more famous locations will generate higher sales.
* **<span style="color:Green">Store Capacity</span>**: Stores with higher capacity will generate higher sales because they will have a diverse range of products and many product units.
* **<span style="color:red">Work Hours</span>**: Stores which have longer working hours will generate higher sales.
* **<span style="color:red">Payment Types</span>**: Stores which are offering many types of payment will generate higher sales.
* **<span style="color:red">Place of payment</span>**: Stores which are offering some kind of recompense if the client is waiting too much will generate higher sales.
* **<span style="color:red">Toys for Kids</span>**: Stores which are offering some types of free toys for the kids when you spent a certain amount of money will generate higher sales.
* **<span style="color:red">Mobile App</span>**: Stores which have a mobile application will generate higher sales because of the convenient to look at the weekly offers.
* **<span style="color:red">Product Returns</span>**: Stores which are offering product returns will generate higher sales because of the emotional sense of loyalty.
* **<span style="color:red">Shopping Type</span>** Stores which are offering online shopping will generate higher sales because of the customers' saved time and convenience.
* **<span style="color:red">Data Analysis, ML</span>**: Stores which are using data analysis and machine learning on their historical data will generate higher sales. 😃
* **<span style="color:red">Demographics</span>**: Based on race, some races can generate higher sales
* **<span style="color:red">Income</span>**: Based on the income, people with higher income can generate higher sales
* **<span style="color:red">Political Vision</span>**: People with are with conservative political vision maybe will generate higher sales 
* **<span style="color:red">Weather</span>**: Based on the weather conditions people can generate higher sales

#### Product Level Hypothesis
* **<span style="color:Green">Visibility</span>**: Better-placed products will generate higher sales because more people are looking at them, so there is a high chance to buy some of them.
* **<span style="color:Green">Product Diversity</span>**: Stores which are selling many types of products (supermarkets) will have higher sales because people will buy many products of them.
* **<span style="color:red">Promotions</span>**: Products which have a promotion will generate higher sales.
* **<span style="color:red">Quality</span>**: Products with higher quality will generate higher sales.
* **<span style="color:red">Brand</span>**: Branded products will generate higher sales because of the trust in the customers.
* **<span style="color:Green">Daily Products</span>**: Stores which are offering daily-used products will generate higher sales because of the high demand.
* **<span style="color:red">Bioproducts</span>** Stores which are offering bioproducts will generate higher sales because people want to be healthy.
* **<span style="color:Green">Price</span>**: Products which have lower prices will generate higher sales.
* **<span style="color:red">Expiration Date of Product</span>**: If the product expiration date is going to expire in 2-3 days are the people going to take this product?

|Variable|Description|Relation to Hypothesis|
|---|---|---|
|Item_Identifier|Unique Product ID|ID Variable|
|Item_Weight|Weight of product|Not considered in hypothesis|
|Item_Fat_Content|Whether the product is low fat or not|Linked to "Daily Products" hypothesis. Low-fat products are used more.|
|Item_Visability|The % of the total display area of all products in a store allocated to the particular product|Linked to "Visibility" hypothesis.|
|Item_Type|The category to which the product belongs|Linked to "Product Diversity" hypothesis|
|Item_MRP|Maximum Retail Price (list price) of the product|Linked to "Price" hypothesis|
|Outlet_Identifier|Unique store ID|ID Variable|
|Outlet_Establishment_Year|The year in which store was established|Not considered in hypothesis|
|Outlet_Size|The size of the store in terms of ground area covered|Considered in the "Store Capacity" hypothesis|
|Outlet_Location_Type|The type of city in which the store is located|Considered in "Location" hypothesis|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|Considered in the "Supermarket" hypothesis|
|Item_Outlet_Sales|Sales of the product in a particular store. This is the outcome variable to be predicted.|Outcome Variable|

### Part 2: Exploratory Data Analysis

In [4]:
train_data = pd.read_csv("Train_Data.csv")
train_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
test_data = pd.read_csv("Test_Data.csv")
test_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [6]:
data = pd.concat([train_data, test_data], ignore_index = True, sort = False)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 12 columns):
Item_Identifier              14204 non-null object
Item_Weight                  11765 non-null float64
Item_Fat_Content             14204 non-null object
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Outlet_Size                  10188 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Type                  14204 non-null object
Item_Outlet_Sales            8523 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.3+ MB


In [8]:
print(train_data.shape, test_data.shape, data.shape)

(8523, 12) (5681, 11) (14204, 12)


In [10]:
data.apply(lambda x: sum(x.isnull()))

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [11]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [12]:
data.apply(lambda x: len(x.unique()))

Item_Identifier               1559
Item_Weight                    416
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      4
Outlet_Location_Type             3
Outlet_Type                      4
Item_Outlet_Sales             3494
dtype: int64

In [13]:
data.Item_Type.value_counts()

Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

## Resources
* [How to Increase Sales Retail](https://www.vendhq.com/blog/how-to-increase-sales-retail/)